# INSTRUCTIONS

# **Case Study**
This is a financial data *scraping* exercise, meaning targeted retrieval of published data in an exploitable format.

We aim to extract the specific fields listed below from the raw annual reports of three companies: **LVMH, Apple, and Tesco**.

You are free to use the provided documents or rely on your own resources.

We expect you to present your approach and results during an interview.

**{...} List of Fields**

*   Company Name
*   Sales
*   Advertising Expense
*   Personnel Expense
*   List of Company Assets (Net, Gross, Accumulated Amortization Breakdown)

**Annual Reports**


*   **LVMH — December 2023**

    https://lvmh-com.cdn.prismic.io/lvmh-com/ZmMq-pm069VX1j0w_financial-documents-lvmh-december-31-2023.pdf
*   **Apple — September 2024**

    https://s2.q4cdn.com/470004039/files/doc_earnings/2024/q4/filing/10-Q4-2024-As-Filed.pdf
*   **Tesco — 2024**

    https://www.tescoplc.com/media/n3jdzlzc/2138002p5rnkc5w2jz46-2024-02-24-t01_preview.xhtml

# **0 - Import Packages**

In [ ]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00


In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')

# calculs et tableaux
import numpy as np
import pandas as pd
from itertools import chain # loop sur deux ranges

#HTML Scraping
from bs4 import BeautifulSoup
import requests


from google.colab import files # ouvre une interface pour charger un fichier (lire tableau à partir d'une image)
import cv2

# Excel
from openpyxl import load_workbook

# PDF Scraping
import re
from PyPDF2 import PdfReader

# **1- General**

URLS

In [ ]:
#LVMH URLs :
LVMH_website = 'https://www.lvmh.com/en/investors/investors-and-analysts'
LVMH_Doc_denregistrement_universel = '/content/deu-2023-vf_vdef.pdf'
LVMH_annual_report = '/content/lvmh_2023-annual-report.pdf' # pas utilisé
LVMH_doc_financiers = '/content/ZmMkBJm069VX1jwZ_documents-financiers-lvmh-31-decembre-2023.pdf'

#Tesco URLs :
TESCO_annual_report_in_html = 'https://www.tescoplc.com/media/n3jdzlzc/2138002p5rnkc5w2jz46-2024-02-24-t01_preview.xhtml'
TESCO_annual_report_in_pdf = '/content/tescos_ar24.pdf'
TESCO_Financials_excel = '/content/tesco-2024-ar-primary-statements.xlsx'

#Apple URLs
APPLE_annual_report = '/content/10-Q4-2024-As-Filed.pdf'

Dicts

In [ ]:
LVMH_dict = { 'Company Name': 'LVMH', 'currency' : '€','Sales' : np.NaN, 'Advertising Expense' : np.NaN , 'Personnel Expense' : np.NaN }
Tesco_dict = { 'Company Name': 'Tesco', 'currency' : '£','Sales' : np.NaN, 'Advertising Expense' : np.NaN , 'Personnel Expense' : np.NaN }
Apple_dict = { 'Company Name': 'Apple', 'currency' : '$','Sales' : np.NaN, 'Advertising Expense' : np.NaN , 'Personnel Expense' : np.NaN }

# **LVMH**

In [ ]:
# LVMH Sales
response = requests.get(LVMH_website)
soup = BeautifulSoup(response.content, 'html.parser')

lol1 = soup.find("tr", class_= "group/alternateRow odd:bg-basic-greyUltraLight border-b border-basic-greyUltraLight" )
# lol = lol1.find("td", id="table_R2eopji979ubtsmhuba_cell_-1")
# print(lol1)
net_sales = lol1.find_all_next("div", class_="max-w-full whitespace-normal")
net_sales_2023 = net_sales[3].get_text(strip=True) #[2] quand ca marchait avec lol
# print(net_sales_2023)

LVMH_dict['Sales']= net_sales_2023 ## add to dict

# comments
# 'Les ventes sont présentées nettes de toutes formes de remises et ristournes. En particulier, les sommes permettant le référencement des produits
# ou correspondant à des accords de participation publicitaire avec le distributeur viennent en déduction des ventes correspondantes.'

# La devise d’établissement des comptes consolidés est l’euro; les comptes des entités utilisant une monnaie fonctionnelle différente sont
# convertis en euros :
#                     • aux cours de clôture pour les postes du bilan;
#                     • aux cours moyens de la période pour les éléments du compte de résultat.

# notation differentes entre les documents

# LVMH Personnel Expense

# Charger le fichier PDF - rapport-de-gestion-du-conseil-dadministration
pdf = PdfReader(LVMH_Doc_denregistrement_universel)

page_125 = pdf.pages[124]
text = page_125.extract_text()
lines = text.split('\n')
# for line in lines:
#     print(line)

match = re.search(r"Total frais de personnel\s+([\d\s,]+)", text)
Total_frais_de_personnel = match.group(1).split()
Total_frais_de_personnel = [' '.join(Total_frais_de_personnel[i:i+2]) for i in range(0, len(Total_frais_de_personnel), 2)][0]
Total_frais_de_personnel = Total_frais_de_personnel.replace(',','').replace(' ',',')
# print(Total_frais_de_personnel)

LVMH_dict['Personnel Expense']= Total_frais_de_personnel ## add to dict

# LVMH Advertising Expense

# Extraire la page souhaitée
page_293 = pdf.pages[292]
text = page_293.extract_text()
lines = text.split('\n')
# for line in lines:
#     print(line)

match = re.search(r"Frais de publicité et de promotion\s+([\d\s]+)", text)
Frais_de_publicite_et_de_promotion = match.group(1).split()
Frais_de_publicite_et_de_promotion = [' '.join(Frais_de_publicite_et_de_promotion[i:i+2]) for i in range(0, len(Frais_de_publicite_et_de_promotion), 2)][0]
Frais_de_publicite_et_de_promotion = Frais_de_publicite_et_de_promotion.replace(' ',',')
# print(Frais_de_publicite_et_de_promotion)

LVMH_dict['Advertising Expense']= Frais_de_publicite_et_de_promotion ## add to dict
LVMH_dict

{'Company Name': 'LVMH',
 'currency': '€',
 'Sales': '86,153',
 'Advertising Expense': '10,221',
 'Personnel Expense': '13,2026'}

In [ ]:
# LVMH List of Company Assets (Net, Gross, Accumulated Amortization Breakdown)

# IMMOBILISATIONS INCORPORELLES

page_256 = pdf.pages[255]
text = page_256.extract_text()
lines =text.split('\n')

# for index, line in enumerate(lines, start=0):  # `start=1` pour commencer à numéroter à 1
#     print(f"{index}: {line}")

# for i in range(57, 63):
#         print(lines[i])

new_text = lines[57:63]

elements = {}
i=1

for element in new_text:
  element = element.split()
  elements[f'element_{i}'] = element
  i+=1

# for key, value in elements.items():
#     print(f"{key}: {value}")

elements['element_1'][1:3]= [' '.join(elements['element_1'][1:3])]
elements['element_1'][3:5]= [' '.join(elements['element_1'][3:5])]
elements['element_1'][2] = elements['element_1'][2].replace('(', '').replace(')', '')
elements['element_1'][4:6]= [' '.join(elements['element_1'][4:6])]
elements['element_1'][5:7]= [' '.join(elements['element_1'][5:7])]
# print(elements['element_1'])

elements['element_2'][1:3]=[' '.join(elements['element_2'][1:3])]
elements['element_2'][2:4]=[' '.join(elements['element_2'][2:4])]
elements['element_2'][2] = elements['element_2'][2].replace('(','').replace(')','')
elements['element_2'][3:5] = [' '.join(elements['element_2'][3:5])]
elements['element_2'][4:6]= [' '.join(elements['element_2'][4:6])]
elements['element_2'][5:7]= [' '.join(elements['element_2'][5:7])]
# print(elements['element_2'])

elements['element_3'][0:3]=[' '.join(elements['element_3'][0:3])]
elements['element_3'][2] = elements['element_3'][2].replace('(','').replace(')','')
# print(elements['element_3'])

elements['element_4'][0:3]=[' '.join(elements['element_4'][0:3])]
elements['element_4'][1:3]=[' '.join(elements['element_4'][1:3])]
elements['element_4'][2:4]=[' '.join(elements['element_4'][2:4])]
elements['element_4'][2] = elements['element_4'][2].replace('(','').replace(')','')
elements['element_4'][3:5]=[' '.join(elements['element_4'][3:5])]
# print(elements['element_4'])

elements['element_5'][1:3] = [' '.join(elements['element_5'][1:3])]
elements['element_5'][2] = elements['element_5'][2].replace('(','').replace(')','')
# print(elements['element_5'])

elements['element_6'][1:3]=[' '.join(elements['element_6'][1:3])]
elements['element_6'][2:4]=[' '.join(elements['element_6'][2:4])]
elements['element_6'][2] = elements['element_6'][2].replace('(','').replace(')','')
elements['element_6'][3:5]=[' '.join(elements['element_6'][3:5])]
elements['element_6'][4:6]= [' '.join(elements['element_6'][4:6])]
elements['element_6'][5:7]= [' '.join(elements['element_6'][5:7])]
# print(elements['element_6'])


df = pd.DataFrame.from_dict(elements, orient='index')
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

df.rename(columns={0: 'Non current assets', 1: 'Gross 2023', 2: 'Depreciation and amortisation 2023', 3: 'Net assets 2023', 4: 'Net assets 2022', 5: 'Net assets 2021'}, inplace=True)

df

,Non current assets,Gross 2023,Depreciation and amortisation 2023,Net assets 2023,Net assets 2022,Net assets 2021
0,Marques,22 297,812,21 485,21 545,20 873
1,Enseignes,3 972,1 636,2 336,2 410,2 285
2,Licences de distribution,115,98,17,23,53
3,"Logiciels, sites internet",3 946,2 912,1 035,926,849
4,Autres,1 568,851,717,528,490
5,Total,31 897,6 309,25 589,25 432,24 551


In [ ]:
# LVMH List of Company Assets (Net, Gross, Accumulated Amortization Breakdown)

# IMMOBILISATIONS CORPORELLES

page_261 = pdf.pages[260]
text = page_261.extract_text()
lines =text.split('\n')

# for index, line in enumerate(lines, start=0):  # `start=1` pour commencer à numéroter à 1
#     print(f"{index}: {line}")

# for i in range(6, 14):
#         print(lines[i])

new_text = lines[6:14]

elements = {}
i=1

for element in new_text:
  element = element.split()
  elements[f'element_{i}'] = element
  i+=1

# for key, value in elements.items():
#     print(f"{key}: {value}")

elements['element_1'][1:3]= [' '.join(elements['element_1'][1:3])]
elements['element_1'][2] = elements['element_1'][2].replace('(', '').replace(')', '')
elements['element_1'][3:5]= [' '.join(elements['element_1'][3:5])]
elements['element_1'][4:6]= [' '.join(elements['element_1'][4:6])]
elements['element_1'][5:7]= [' '.join(elements['element_1'][5:7])]
# print(elements['element_1'])

elements['element_2'][0:5]=[' '.join(elements['element_2'][0:5])]
elements['element_2'] = [item for item in elements['element_2'] if item != '(a)']
elements['element_2'][1:3]=[' '.join(elements['element_2'][1:3])]
elements['element_2'][2] = elements['element_2'][2].replace('(','').replace(')','')
elements['element_2'][3:5] = [' '.join(elements['element_2'][3:5])]
elements['element_2'][4:6]= [' '.join(elements['element_2'][4:6])]
elements['element_2'][5:7]= [' '.join(elements['element_2'][5:7])]
# print(elements['element_2'])

elements['element_3'][1:3]=[' '.join(elements['element_3'][1:3])]
elements['element_3'][2:4]=[' '.join(elements['element_3'][2:4])]
elements['element_3'][2] = elements['element_3'][2].replace('(','').replace(')','')
elements['element_3'][3:5]=[' '.join(elements['element_3'][3:5])]
elements['element_3'][4:6]=[' '.join(elements['element_3'][4:6])]
elements['element_3'][5:7]=[' '.join(elements['element_3'][5:7])]
# print(elements['element_3'])

elements['element_4'][0:2]=[' '.join(elements['element_4'][0:2])]
elements['element_4'][2] = elements['element_4'][2].replace('(','').replace(')','')
# print(elements['element_4'])

elements['element_5'][0:4] = [' '.join(elements['element_5'][0:4])]
elements['element_5'][1:3] = [' '.join(elements['element_5'][1:3])]
elements['element_5'][2:4] = [' '.join(elements['element_5'][2:4])]
elements['element_5'][2] = elements['element_5'][2].replace('(','').replace(')','')
elements['element_5'][3:5]=[' '.join(elements['element_5'][3:5])]
elements['element_5'][4:6]=[' '.join(elements['element_5'][4:6])]
elements['element_5'][5:7]=[' '.join(elements['element_5'][5:7])]
# print(elements['element_5'])

elements['element_6'][0:3]=[' '.join(elements['element_6'][0:3])]
elements['element_6'][1:3]=[' '.join(elements['element_6'][1:3])]
elements['element_6'][2] = elements['element_6'][2].replace('(','').replace(')','')
elements['element_6'][3:5]=[' '.join(elements['element_6'][3:5])]
elements['element_6'][4:6]= [' '.join(elements['element_6'][4:6])]
elements['element_6'][5:7]= [' '.join(elements['element_6'][5:7])]
# print(elements['element_6'])

elements['element_7'][0:3]=[' '.join(elements['element_7'][0:3])]
elements['element_7'][1:3]=[' '.join(elements['element_7'][1:3])]
elements['element_7'][2] = elements['element_7'][2].replace('(','').replace(')','')
elements['element_7'][3:5]=[' '.join(elements['element_7'][3:5])]
elements['element_7'][4:6]= [' '.join(elements['element_7'][4:6])]
elements['element_7'][5:7]= [' '.join(elements['element_7'][5:7])]
# print(elements['element_7'])

elements['element_8'][1:3]=[' '.join(elements['element_8'][1:3])]
elements['element_8'][2:4] = [' '.join(elements['element_8'][2:4])]
elements['element_8'][2] = elements['element_8'][2].replace('(','').replace(')','')
elements['element_8'][3:5]=[' '.join(elements['element_8'][3:5])]
elements['element_8'][4:6]= [' '.join(elements['element_8'][4:6])]
elements['element_8'][5:7]= [' '.join(elements['element_8'][5:7])]
# print(elements['element_8'])

df = pd.DataFrame.from_dict(elements, orient='index')
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

df.rename(columns={0: 'Non current assets', 1: 'Gross 2023', 2: 'Depreciation and amortisation 2023', 3: 'Net assets 2023', 4: 'Net assets 2022', 5: 'Net assets 2021'}, inplace=True)

df

,Non current assets,Gross 2023,Depreciation and amortisation 2023,Net assets 2023,Net assets 2022,Net assets 2021
0,Terrains,7 972,22,7 950,5 511,4 804
1,Terres à vignes et vignobles,3 084,136,2 948,2 729,2 623
2,Constructions,8 318,3 055,5 263,4 823,4 145
3,Immeubles locatifs,366,51,316,434,321
4,"Agencements, matériels et installations",20 880,14 227,6 653,5 773,5 114
5,Immobilisations en- cours,2 125,45,2 080,1 809,1 302
6,Autres immobilisations corporelles,2 719,598,2 121,1 977,1 886
7,Total,45 465,18 135,27 331,23 055,20 193


In [ ]:
# LVMH List of Company Assets (Net, Gross, Accumulated Amortization Breakdown)

pdf = PdfReader(LVMH_doc_financiers)

page_26 = pdf.pages[25]
text = page_26.extract_text()
lines =text.split('\n')

# for index, line in enumerate(lines, start=0):  # `start=1` pour commencer à numéroter à 1
#     print(f"{index}: {line}")

# for i in range(6, 15):
#          print(lines[i])

new_text = lines[6:15]

elements = {}
i=1

for element in new_text:
  element = element.split()
  elements[f'element_{i}'] = element
  i+=1

# for key, value in elements.items():
#     print(f"{key}: {value}")

for i in range(1, 10):
    t = 0
    for item in elements[f'element_{i}']:
        if isinstance(item, str) and not item.isdigit():
            t += 1
    elements[f'element_{i}'][0:t]=[' '.join(elements[f'element_{i}'][0:t])]

for i in range(1,10) :
  if i == 5 :
    # print(i)
    elements[f'element_{i}'] = [item for item in elements[f'element_{i}'] if item != elements[f'element_{i}'][1]]
    elements[f'element_{i}'][2:4]=[' '.join(elements[f'element_{i}'][2:4])]
    elements[f'element_{i}'][3:5]= [' '.join(elements[f'element_{i}'][3:5])]
    # print(elements[f'element_{i}'])
  elif (i == 8 or i== 9) :
    # print(i)
    elements[f'element_{i}'][1:3]=[' '.join(elements[f'element_{i}'][1:3])]
    elements[f'element_{i}'][2:4]=[' '.join(elements[f'element_{i}'][2:4])]
    elements[f'element_{i}'][3:5]= [' '.join(elements[f'element_{i}'][3:5])]
    # print(elements[f'element_{i}'])
  else:
    # print(i)
    elements[f'element_{i}'] = [item for item in elements[f'element_{i}'] if item != elements[f'element_{i}'][1]]
    elements[f'element_{i}'][1:3]=[' '.join(elements[f'element_{i}'][1:3])]
    elements[f'element_{i}'][2:4]=[' '.join(elements[f'element_{i}'][2:4])]
    elements[f'element_{i}'][3:5]= [' '.join(elements[f'element_{i}'][3:5])]
    # print(elements[f'element_{i}'])

df = pd.DataFrame.from_dict(elements, orient='index')
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

df.rename(columns={0: 'Non current assets', 1:'Net assets 2023', 2: 'Net assets 2022', 3: 'Net assets 2021'}, inplace=True)

df

# comment :
# Écarts d’acquisition : page 258 de deu-2023-vf_vdef.pdf, gross amortisation and net sur 2023 2022 2021
# Droits d’utilisation : page 263 de deu-2023-vf_vdef.pdf, gross amortisation and net sur 2023 2022 2021
# Participations mises en équivalence : page 267 de deu-2023-vf_vdef.pdf, que des valeurs net sur 2023 2022 2021
# Investissements financiers : page 267 de deu-2023-vf_vdef.pdf, que des valeurs net sur 2023 2022 2021
# Autres actifs non courants : page 268 de deu-2023-vf_vdef.pdf, que des valeurs net sur 2023 2022 2021

# Actifs non courants = Total

,Non current assets,Net assets 2023,Net assets 2022,Net assets 2021
0,Marques et autres immobilisations incorporelles,25 589,25 432,24 551
1,Écarts d’acquisition,24 022,24 782,25 904
2,Immobilisations corporelles,27 331,23 055,20 193
3,Droits d’utilisation,15 679,14 615,13 705
4,Participations mises en équivalence,991,1 066,1 084
5,Investissements financiers,1 363,1 109,1 363
6,Autres actifs non courants,1 017,1 186,1 054
7,Impôts différés,3 992,3 661,3 156
8,Actifs non courants,99 984,94 906,91 010


In [ ]:
# LVMH current assets

pdf = PdfReader(LVMH_Doc_denregistrement_universel)

page_242 = pdf.pages[241]
text = page_242.extract_text()
lines =text.split('\n')

# for index, line in enumerate(lines, start=0):  # `start=1` pour commencer à numéroter à 1
#     print(f"{index}: {line}")

# for i in range(15, 21):
#          print(lines[i])

new_text = lines[15:21]

elements = {}
i=1

for element in new_text:
  element = element.split()
  elements[f'element_{i}'] = element
  i+=1

# for key, value in elements.items():
#     print(f"{key}: {value}")

for i in range(1, 7):
    t = 0
    for item in elements[f'element_{i}']:
        if isinstance(item, str) and not item.isdigit():
            t += 1
    elements[f'element_{i}'][0:t]=[' '.join(elements[f'element_{i}'][0:t])]

# for key, value in elements.items():
#     print(f"{key}: {value}")

for i in range(1,7) :
  if i == 3 :
    # print(i)
    # print(elements[f'element_{i}'])
    pass
  elif i== 6 :
    # print(i)
    elements[f'element_{i}'][1:3]=[' '.join(elements[f'element_{i}'][1:3])]
    elements[f'element_{i}'][2:4]=[' '.join(elements[f'element_{i}'][2:4])]
    elements[f'element_{i}'][3:5]= [' '.join(elements[f'element_{i}'][3:5])]
    # print(elements[f'element_{i}'])
  else:
    # print(i)
    elements[f'element_{i}'] = [item for item in elements[f'element_{i}'] if item != elements[f'element_{i}'][1]]
    elements[f'element_{i}'][1:3]=[' '.join(elements[f'element_{i}'][1:3])]
    elements[f'element_{i}'][2:4]=[' '.join(elements[f'element_{i}'][2:4])]
    elements[f'element_{i}'][3:5]= [' '.join(elements[f'element_{i}'][3:5])]
    # print(elements[f'element_{i}'])

df = pd.DataFrame.from_dict(elements, orient='index')
df.reset_index(inplace=True)
df.drop(columns=['index'], inplace=True)

df.rename(columns={0: 'Current assets', 1:'Net assets 2023', 2: 'Net assets 2022', 3: 'Net assets 2021'}, inplace=True)

df

,Current assets,Net assets 2023,Net assets 2022,Net assets 2021
0,Stocks et en -cours,22 952,20 319,16 549
1,Créances clients et comptes rattachés,4 728,4 258,3 787
2,Impôts sur les résultats,533,375,338
3,Autres actifs courants,7 723,7 488,5 606
4,Trésorerie et équivalents de trésorerie,7 774,7 300,8 021
5,Actifs courants,43 710,39 740,34 301


# **TESCO**

In [ ]:
# Tesco Sales

TESCO_Financials_excel

workbook = load_workbook(TESCO_Financials_excel)
sheet = workbook['Income Statement']
cell_value = sheet['F8'].value
cell_value = "{:,}".format(cell_value) # "{:,}" séparateurs de milliers
Tesco_dict['Sales']= cell_value ## add to dict
Tesco_dict

# Tesco Personnel Expense

# Charger le fichier PDF
pdf_Tesco = PdfReader(TESCO_annual_report_in_pdf)

# Extraire la page souhaitée
page = pdf_Tesco.pages[146]
text = page.extract_text()
lines = text.split('\n')
# for line in lines:
#     print(line)

match = re.search(r"Total\s+([\d,]+)", text)
total_Personnel_Expense_2024 = match.group(1)  # Récupérer la première valeur
print(total_Personnel_Expense_2024)

Tesco_dict['Personnel Expense']= total_Personnel_Expense_2024 ## add to dict
Tesco_dict

# #si j'ai temps distinguer discontinued operation de total operations

# # Tesco Advertising Expenses
# # absentes

# comments
# le terme "Net Sales" n'est pas toujours explicitement mentionné dans les états financiers britanniques,
# mais "Revenue from sale of goods and services" est l'équivalent.

# Personnel expense for discontinued operation = : 0,157, total continued operations : 8,004 page 147

8,161


{'Company Name': 'Tesco',
 'currency': '£',
 'Sales': '67,673',
 'Advertising Expense': nan,
 'Personnel Expense': '8,161'}

In [ ]:
# tesco List of Company Assets (Net, Gross, Accumulated Amortization Breakdown)

workbook = load_workbook(TESCO_Financials_excel)

sheet_Balance_Sheet = workbook['Balance Sheet']

List_non_current_assets_name = []
List_non_current_assets_val_2024 = []
List_non_current_assets_val_2023 = []
List_non_current_assets_val_2022 = []

total_2024 = 0
total_2023 = 0
total_2022 = 0

for i in range(6,18):
  cell_non_current_assets_name = sheet_Balance_Sheet[f'B{i}'].value
  List_non_current_assets_name.append(cell_non_current_assets_name)

  cell_non_current_assets_val_2024 = sheet_Balance_Sheet[f'D{i}'].value
  cell_non_current_assets_val_2023 = sheet_Balance_Sheet[f'E{i}'].value
  cell_non_current_assets_val_2022 = sheet_Balance_Sheet[f'F{i}'].value

  if cell_non_current_assets_val_2024 == '–':
    cell_non_current_assets_val_2024 = 0

  if cell_non_current_assets_val_2023 == '–':
    cell_non_current_assets_val_2023 = 0

  if cell_non_current_assets_val_2022 == '–':
    cell_non_current_assets_val_2022 = 0

  List_non_current_assets_val_2024.append(cell_non_current_assets_val_2024)
  List_non_current_assets_val_2023.append(cell_non_current_assets_val_2023)
  List_non_current_assets_val_2022.append(cell_non_current_assets_val_2022)

  total_2024 += cell_non_current_assets_val_2024
  total_2023 += cell_non_current_assets_val_2023
  total_2022 += cell_non_current_assets_val_2022


List_non_current_assets_name.append('Total Non-current assets')
List_non_current_assets_val_2024.append(total_2024)
List_non_current_assets_val_2023.append(total_2023)
List_non_current_assets_val_2022.append(total_2022)

print("Liste des non current assets nom : ",List_non_current_assets_name)
print("Liste des non current assets valeurs 2024 : ",List_non_current_assets_val_2024)
print("Liste des non current assets valeurs 2023 : ",List_non_current_assets_val_2023)
print("Liste des non current assets valeurs 2022 : ",List_non_current_assets_val_2022)

cell_Net_assets_name = sheet_Balance_Sheet['B54'].value
cell_value_Net_assets_val_2024 = sheet_Balance_Sheet['D54'].value
cell_value_Net_assets_val_2023 = sheet_Balance_Sheet['E54'].value
cell_value_Net_assets_val_2022 = sheet_Balance_Sheet['F54'].value


sheet_Cash_Flow = workbook['Cash Flow']
cell_amortisation_name = sheet_Cash_Flow['B8'].value
cell_value_amortisation_val_2024 = sheet_Cash_Flow['D8'].value
cell_value_amortisation_val_2023 = sheet_Cash_Flow['E8'].value

# Charger le fichier PDF
pdf_Tesco = PdfReader(TESCO_annual_report_in_pdf)

# Extraire la page souhaitée
page = pdf_Tesco.pages[143]
text = page.extract_text()
lines = text.split('\n')

# Texte donné (à remplacer par votre extraction PDF si nécessaire)

start_index_2024 = text.find("Depreciation and amortisation:")
end_index_2024 = text.find("Impairment:")

# Si trouvé, couper le texte à partir de cette position
if start_index_2024 != -1 and end_index_2024 != -1:
    new_text_2024 = text[start_index_2024:end_index_2024]

# Initialiser les listes
List_Depreciation_and_amortisation_name_2024 = []
List_Depreciation_and_amortisation_name_2023 = []

List_Depreciation_and_amortisation_val_2024 = []
List_Depreciation_and_amortisation_val_2023 = []

# Diviser le texte en lignes
lines_2024 = new_text_2024.split('\n')

# for index, line in enumerate(lines, start=0):  # `start=1` pour commencer à numéroter à 1
#     print(f"{index}: {line}")

# for i in range(92, 97):
#         print(lines[i])

lines_2023 = lines[92:97]

# Extraire les noms d'actifs et les valeurs avec des expressions régulières

for line in lines_2024 :
    # Find the name of the asset
    name_match = re.match(r'([A-Za-z\s,]+)', line)
    if name_match:
        name = name_match.group(1).strip()

        # Find all numbers within parentheses
        values = re.findall(r'\((\d+)\)', line)
        if values:
            # Get the last value
            last_value = int(values[-1])

            # Add to the lists
            List_Depreciation_and_amortisation_name_2024.append(name)
            List_Depreciation_and_amortisation_val_2024.append(last_value)

List_Depreciation_and_amortisation_name_2024.append(cell_amortisation_name)
List_Depreciation_and_amortisation_val_2024.append(cell_value_amortisation_val_2024)

for line in lines_2023 :
    # Find the name of the asset
    name_match = re.match(r'([A-Za-z\s,]+)', line)
    if name_match:
        name = name_match.group(1).strip()

        # Find all numbers within parentheses
        values = re.findall(r'\((\d+)\)', line)
        if values:
            # Get the last value
            last_value = int(values[-1])

            # Add to the lists
            List_Depreciation_and_amortisation_name_2023.append(name)
            List_Depreciation_and_amortisation_val_2023.append(last_value)

List_Depreciation_and_amortisation_name_2023.append(cell_amortisation_name)
List_Depreciation_and_amortisation_val_2023.append(cell_value_amortisation_val_2023)

# Afficher les résultats
print("Liste des noms d'amortissement 2024 : ", List_Depreciation_and_amortisation_name_2024)
print("Liste des valeurs d'amortissement 2024 : ", List_Depreciation_and_amortisation_val_2024)

print("Liste des noms d'amortissement 2023 : ", List_Depreciation_and_amortisation_name_2023)
print("Liste des valeurs d'amortissement 2023 : ", List_Depreciation_and_amortisation_val_2023)

Liste des non current assets nom :  ['Goodwill and other intangible assets', 'Property, plant and equipment', 'Right of use assets', 'Investment property', 'Investments in joint ventures and associates', 'Other investments', 'Trade and other receivables', 'Loans and advances to customers', 'Reinsurance contract assets', 'Derivative financial instruments', 'Post-employment benefit surplus', 'Deferred tax assets', 'Total Non-current assets']
Liste des non current assets valeurs 2024 :  [5066, 17221, 5478, 24, 102, 1546, 36, 0, 125, 781, 22, 32, 30433]
Liste des non current assets valeurs 2023 :  [5375, 16862, 5500, 24, 93, 1339, 79, 3029, 135, 873, 6, 84, 33399]
Liste des non current assets valeurs 2022 :  [5360, 17060, 5720, 22, 86, 1253, 159, 3141, 171, 942, 3150, 88, 37152]
Liste des noms d'amortissement 2024 :  ['Property, plant and equipment', 'Right of use assets', 'Other intangible assets', 'Depreciation and amortisation']
Liste des valeurs d'amortissement 2024 :  [896, 543, 255, 

In [ ]:
df1_2024 = pd.DataFrame({
    'non current assets': List_Depreciation_and_amortisation_name_2024,
    'Depreciation and amortisation 2024': List_Depreciation_and_amortisation_val_2024
})

df1_2024.loc[df1_2024['non current assets'] == 'Depreciation and amortisation', 'non current assets'] = 'Total Non-current assets'

df1_2023 = pd.DataFrame({
    'non current assets': List_Depreciation_and_amortisation_name_2023,
    'Depreciation and amortisation 2023': List_Depreciation_and_amortisation_val_2023
})

df1_2023.loc[df1_2023['non current assets'] == 'Depreciation and amortisation', 'non current assets'] = 'Total Non-current assets'

df1 = pd.merge(df1_2024, df1_2023, on='non current assets', how='outer')

df2 = pd.DataFrame({
    'non current assets': List_non_current_assets_name,
    'Gross 2024': List_non_current_assets_val_2024,
    'Gross 2023': List_non_current_assets_val_2023,
    'Gross 2022': List_non_current_assets_val_2022
})

# Utiliser merge pour combiner les deux DataFrames avec une union externe
df_combined = pd.merge(df1, df2, on='non current assets', how='outer')
df_combined.iloc[-2:] = df_combined.iloc[[-1, -2]]

cell_Net_assets_name = sheet_Balance_Sheet[f'B54'].value
cell_Net_assets_val_2024 = sheet_Balance_Sheet[f'D54'].value
cell_Net_assets_val_2023 = sheet_Balance_Sheet[f'E54'].value
cell_Net_assets_val_2022 = sheet_Balance_Sheet[f'F54'].value

L_Net_assets_2024 = ['NaN' for i in range(len(df_combined)-1)]
L_Net_assets_2023 = ['NaN' for i in range(len(df_combined)-1)]
L_Net_assets_2022 = ['NaN' for i in range(len(df_combined)-1)]

L_Net_assets_2024.insert(len(df_combined),cell_Net_assets_val_2024)
L_Net_assets_2023.insert(len(df_combined),cell_Net_assets_val_2023)
L_Net_assets_2022.insert(len(df_combined),cell_Net_assets_val_2022)

# # Afficher le DataFrame combiné
df_combined['Net assets 2024'] = L_Net_assets_2024
df_combined['Net assets 2023'] = L_Net_assets_2023
df_combined['Net assets 2022'] = L_Net_assets_2022

# Réorganiser les colonnes dans l'ordre souhaité
df_combined = df_combined[['non current assets', 'Gross 2024', 'Depreciation and amortisation 2024', 'Net assets 2024', 'Gross 2023', 'Depreciation and amortisation 2023', 'Net assets 2023', 'Gross 2022', 'Net assets 2022']]
df_combined

#comment :
#pb gross - amortisation != net

,non current assets,Gross 2024,Depreciation and amortisation 2024,Net assets 2024,Gross 2023,Depreciation and amortisation 2023,Net assets 2023,Gross 2022,Net assets 2022
0,Deferred tax assets,32.0,NaN,NaN,84.0,NaN,NaN,88.0,NaN
1,Derivative financial instruments,781.0,NaN,NaN,873.0,NaN,NaN,942.0,NaN
2,Goodwill and other intangible assets,5066.0,NaN,NaN,5375.0,NaN,NaN,5360.0,NaN
3,Investment property,24.0,NaN,NaN,24.0,1.0,NaN,22.0,NaN
4,Investments in joint ventures and associates,102.0,NaN,NaN,93.0,NaN,NaN,86.0,NaN
5,Loans and advances to customers,0.0,NaN,NaN,3029.0,NaN,NaN,3141.0,NaN
6,Other intangible assets,NaN,255.0,NaN,NaN,247.0,NaN,NaN,NaN
7,Other investments,1546.0,NaN,NaN,1339.0,NaN,NaN,1253.0,NaN
8,Post-employment benefit surplus,22.0,NaN,NaN,6.0,NaN,NaN,3150.0,NaN
9,"Property, plant and equipment",17221.0,896.0,NaN,16862.0,880.0,NaN,17060.0,NaN


In [ ]:
# Tesco current assets

workbook = load_workbook(TESCO_Financials_excel)

sheet_Balance_Sheet = workbook['Balance Sheet']

List_current_assets_name = []
List_current_assets_val_2024 = []
List_current_assets_val_2023 = []
List_current_assets_val_2022 = []

total_2024 = 0
total_2023 = 0
total_2022 = 0

for i in range(20,28):
  cell_current_assets_name = sheet_Balance_Sheet[f'B{i}'].value
  List_current_assets_name.append(cell_current_assets_name)

  cell_current_assets_val_2024 = sheet_Balance_Sheet[f'D{i}'].value
  cell_current_assets_val_2023 = sheet_Balance_Sheet[f'E{i}'].value
  cell_current_assets_val_2022 = sheet_Balance_Sheet[f'F{i}'].value

  if cell_current_assets_val_2024 == '–':
    cell_current_assets_val_2024 = 0

  if cell_current_assets_val_2023 == '–':
    cell_current_assets_val_2023 = 0

  if cell_current_assets_val_2022 == '–':
    cell_current_assets_val_2022 = 0

  List_current_assets_val_2024.append(cell_current_assets_val_2024)
  List_current_assets_val_2023.append(cell_current_assets_val_2023)
  List_current_assets_val_2022.append(cell_current_assets_val_2022)

  total_2024 += cell_current_assets_val_2024
  total_2023 += cell_current_assets_val_2023
  total_2022 += cell_current_assets_val_2022


List_current_assets_name.append('Total current assets')
List_current_assets_val_2024.append(total_2024)
List_current_assets_val_2023.append(total_2023)
List_current_assets_val_2022.append(total_2022)

print("Liste des non current assets nom : ",List_current_assets_name)
print("Liste des non current assets valeurs 2024 : ",List_current_assets_val_2024)
print("Liste des non current assets valeurs 2023 : ",List_current_assets_val_2023)
print("Liste des non current assets valeurs 2022 : ",List_current_assets_val_2022)

df3= pd.DataFrame({
    'non current assets': List_current_assets_name,
    'Net 2024': List_current_assets_val_2024,
    'Net 2023': List_current_assets_val_2023,
    'Net 2022': List_current_assets_val_2022
})

df3

Liste des non current assets nom :  ['Other investments', 'Inventories', 'Trade and other receivables', 'Loans and advances to customers', 'Derivative financial instruments', 'Current tax assets', 'Short-term investments', 'Cash and cash equivalents', 'Total current assets']
Liste des non current assets valeurs 2024 :  [206, 2635, 1349, 0, 55, 110, 2128, 2340, 8823]
Liste des non current assets valeurs 2023 :  [353, 2510, 1235, 3948, 57, 63, 1628, 2465, 12259]
Liste des non current assets valeurs 2022 :  [226, 2339, 1218, 3251, 69, 93, 2076, 2345, 11617]


,non current assets,Net 2024,Net 2023,Net 2022
0,Other investments,206,353,226
1,Inventories,2635,2510,2339
2,Trade and other receivables,1349,1235,1218
3,Loans and advances to customers,0,3948,3251
4,Derivative financial instruments,55,57,69
5,Current tax assets,110,63,93
6,Short-term investments,2128,1628,2076
7,Cash and cash equivalents,2340,2465,2345
8,Total current assets,8823,12259,11617


# **APPLE**

In [ ]:
# Apple sales

# Charger le fichier PDF
pdf = PdfReader(APPLE_annual_report)

# Extraire la page souhaitée
page_24 = pdf.pages[24]
text = page_24.extract_text()

# Utiliser une expression régulière pour extraire "Total net sales" pour 2024
match = re.search(r"Total net sales \$ ([\d,]+)", text)
total_net_sales_2024 = match.group(1)  # Récupérer la première valeur
# print(total_net_sales_2024)

Apple_dict['Sales']= total_net_sales_2024 ## add to dict
Apple_dict

# Apple Selling, general and administrative
page_27 = pdf.pages[26]
text = page_27.extract_text()
# for line in lines :
#   print(line)

lines = text.split('\n')
match = re.search(r"Selling, general and administrative\s+\$\s+([\d,]+)", text)
total_Selling_general_and_administrative = match.group(1)
# print(total_Selling_general_and_administrative)

Apple_dict['SG&A']= total_Selling_general_and_administrative
Apple_dict

{'Company Name': 'Apple',
 'currency': '$',
 'Sales': '391,035',
 'Advertising Expense': nan,
 'Personnel Expense': nan,
 'SG&A': '26,097'}

In [ ]:
# Apple List of Company Assets (Net, Gross, Accumulated Amortization Breakdown)

# Charger le fichier PDF
pdf_Apple = PdfReader(APPLE_annual_report)

# Extraire la page souhaitée
page_34 = pdf_Apple.pages[33]
text = page_34.extract_text()
lines = text.split('\n')

# for index, line in enumerate(lines, start=0):  # `start=1` pour commencer à numéroter à 1
#     print(f"{index}: {line}")

# for i in range(16, 20):
#         print(lines[i])

new_text = lines[16:20]

elements = {}
i=1

for element in new_text:
  element = element.split()
  elements[f'element_{i}'] = element
  elements[f'element_{i}'] =  elements[f'element_{i}']
  elements[f'element_{i}'][0:(len(elements[f'element_{i}'])-2)] = [' '.join(elements[f'element_{i}'][0:(len(elements[f'element_{i}'])-2)])]
  # print(elements[f'element_{i}'])
  i+=1

df1 = pd.DataFrame.from_dict(elements, orient='index')
df1.reset_index(inplace=True)
df1.drop(columns=['index'], inplace=True)

df1.rename(columns={0: 'Non current assets', 1: 'Gross 2024', 2: 'Gross 2023'}, inplace=True)

# Supprimer les lignes qui contiennent 'Property, plant and equipment, net' et 'Other non-current assets' dans la colonne "Non current assets"
df1 = df1[~df1['Non current assets'].isin(['Property, plant and equipment, net', 'Other non-current assets'])]

df1.reset_index(drop=True, inplace=True)

df1

,Non current assets,Gross 2024,Gross 2023
0,Marketable securities,"91,479","100,544"
1,Total non-current assets,"211,993","209,017"


In [ ]:
# Charger le fichier PDF
pdf_Apple = PdfReader(APPLE_annual_report)

# Extraire la page souhaitée
page_42 = pdf_Apple.pages[41]
text = page_42.extract_text()
lines = text.split('\n')

# for index, line in enumerate(lines, start=0):  # `start=1` pour commencer à numéroter à 1
#     print(f"{index}: {line}")

# for i in chain(range(4, 10), range(15, 20)): #from itertools
#     print(lines[i])

new_text = lines[4:10] + lines[17:20]

elements = {}
i=1

for element in new_text:
  element = element.split()
  elements[f'element_{i}'] = element
  # print(print(elements[f'element_{i}']))
  if '$' in elements[f'element_{i}']:
    elements[f'element_{i}'] = [item for item in elements[f'element_{i}'] if item != '$']
  elements[f'element_{i}'][0:(len(elements[f'element_{i}'])-2)] = [' '.join(elements[f'element_{i}'][0:(len(elements[f'element_{i}'])-2)])]
  # print(elements[f'element_{i}'])
  i+=1

elements['element_4'][0]=' Total property, plant and equipment'
elements['element_5'][1]= elements['element_5'][1].replace('(','').replace(')','')
elements['element_5'][2]= elements['element_5'][2].replace('(','').replace(')','')

# creer liste de clef 'elements['element_5'][0]' + '_2024' avec des NaN partout sauf à la ligne 'Total property, plant and equipment' qui a 'elements['element_5'][1]
Accumulated_depreciation_2024 = [np.nan] * 6
Accumulated_depreciation_2024[3] = elements['element_5'][1]

Accumulated_depreciation_2023 = [np.nan] * 6
Accumulated_depreciation_2023[3] = elements['element_5'][2]

Net_2024 = [np.nan] * 6
Net_2024[3] = elements['element_6'][1]

Net_2023 = [np.nan] * 6
Net_2023[3] = elements['element_6'][2]

dict_acc_et_net = {
    'Depreciation and amortization 2024': Accumulated_depreciation_2024,
    'Depreciation and amortization 2023': Accumulated_depreciation_2023,
    'Net 2024': Net_2024,
    'Net 2023': Net_2023
}


elements.pop('element_5', None) # amortissement
elements.pop('element_6', None) # net
elements.pop('element_9', None) # Total other non-current assets

df2 = pd.DataFrame.from_dict(elements, orient='index')
df2.reset_index(inplace=True)
df2.drop(columns=['index'], inplace=True)

df2.rename(columns={0: 'Non current assets', 1: 'Gross 2024', 2: 'Gross 2023'}, inplace=True)


# Ajouter les nouvelles colonnes au DataFrame
for col_name, col_values in dict_acc_et_net.items():
    df2[col_name] = col_values

# Afficher le DataFrame mis à jour

df2

,Non current assets,Gross 2024,Gross 2023,Depreciation and amortization 2024,Depreciation and amortization 2023,Net 2024,Net 2023
0,Land and buildings,"24,690","23,446",NaN,NaN,NaN,NaN
1,"Machinery, equipment and internal-use software","80,205","78,314",NaN,NaN,NaN,NaN
2,Leasehold improvements,"14,233","12,839",NaN,NaN,NaN,NaN
3,"Total property, plant and equipment","119,128","114,599","73,448","70,884","45,680","43,715"
4,Deferred tax assets,"19,499","17,852",NaN,NaN,NaN,NaN
5,Other non-current assets,"55,335","46,906",NaN,NaN,NaN,NaN


In [ ]:
df_combined = pd.merge(df1, df2, on=['Non current assets', 'Gross 2024', 'Gross 2023'], how='outer')

# Charger le fichier PDF
pdf_Apple = PdfReader(APPLE_annual_report)

# Extraire la page souhaitée
page_36 = pdf_Apple.pages[35]
text = page_36.extract_text()
lines = text.split('\n')

# for index, line in enumerate(lines, start=0):  # `start=1` pour commencer à numéroter à 1
#     print(f"{index}: {line}")

new_text = lines[13]
new_text = new_text.split()
new_text[0:3] = [' '.join(new_text[0:3])]

# print(new_text)

df_combined.loc[df_combined['Non current assets'] == 'Total non-current assets', 'Depreciation and amortization 2024'] = new_text[1]
df_combined.loc[df_combined['Non current assets'] == 'Total non-current assets', 'Depreciation and amortization 2023'] = new_text[2]

df_combined.iloc[1], df_combined.iloc[4] = df_combined.iloc[4].copy(), df_combined.iloc[1].copy()
df_combined.iloc[4], df_combined.iloc[5] = df_combined.iloc[5].copy(), df_combined.iloc[4].copy()
df_combined.reset_index(drop=True, inplace=True)

#comments
df_combined = df_combined[['Non current assets', 'Gross 2024', 'Depreciation and amortization 2024', 'Net 2024', 'Gross 2023', 'Depreciation and amortization 2023', 'Net 2023']]
df_combined
# 'Machinery, equipment and internal-use software', 'Land and buildings', 'Leasehold improvements' sont les sous catégories
#  de 'Total non-current assets'

# dans le rapport annuel d'apple : Gross de 'Total non-current assets' est calculé comme la somme de
# Gross 'Other non-current assets' , Gross 'Deferred tax assets', Gross 'Marketable securities' ET Net 'Total property, plant and equipment'


,Non current assets,Gross 2024,Depreciation and amortization 2024,Net 2024,Gross 2023,Depreciation and amortization 2023,Net 2023
0,"Total property, plant and equipment","119,128","73,448","45,680","114,599","70,884","43,715"
1,"Machinery, equipment and internal-use software","80,205",NaN,NaN,"78,314",NaN,NaN
2,Land and buildings,"24,690",NaN,NaN,"23,446",NaN,NaN
3,Leasehold improvements,"14,233",NaN,NaN,"12,839",NaN,NaN
4,Marketable securities,"91,479",NaN,NaN,"100,544",NaN,NaN
5,Deferred tax assets,"19,499",NaN,NaN,"17,852",NaN,NaN
6,Other non-current assets,"55,335",NaN,NaN,"46,906",NaN,NaN
7,Total non-current assets,"211,993","11,445",NaN,"209,017","11,519",NaN


In [ ]:
# Apple assets

# Charger le fichier PDF
pdf_Apple = PdfReader(APPLE_annual_report)

# Extraire la page souhaitée
page_34 = pdf_Apple.pages[33]
text = page_34.extract_text()
lines = text.split('\n')

# for index, line in enumerate(lines, start=0):  # `start=1` pour commencer à numéroter à 1
#     print(f"{index}: {line}")

new_text = lines[8:15]

# for line in new_text :
#   print(line)

elements = {}
i=1

for element in new_text:
  element = element.split()
  elements[f'element_{i}'] = element
  # print(print(elements[f'element_{i}']))
  if '$' in elements[f'element_{i}']:
    elements[f'element_{i}'] = [item for item in elements[f'element_{i}'] if item != '$']
  elements[f'element_{i}'][0:(len(elements[f'element_{i}'])-2)] = [' '.join(elements[f'element_{i}'][0:(len(elements[f'element_{i}'])-2)])]
  # print(elements[f'element_{i}'])
  i+=1

# for key, value in elements.items():
#      print(f"{key}: {value}")

df1 = pd.DataFrame.from_dict(elements, orient='index')
df1.reset_index(inplace=True)
df1.drop(columns=['index'], inplace=True)

df1.rename(columns={0: 'Current assets', 1: 'Net 2024', 2: 'Net 2023'}, inplace=True)

df1

,Current assets,Net 2024,Net 2023
0,Cash and cash equivalents,"29,943","29,965"
1,Marketable securities,"35,228","31,590"
2,"Accounts receivable, net","33,410","29,508"
3,Vendor non-trade receivables,"32,833","31,477"
4,Inventories,"7,286","6,331"
5,Other current assets,"14,287","14,695"
6,Total current assets,"152,987","143,566"


# **Brouillon**

In [ ]:
# Charger le contenu HTML de la page
url = 'https://www.tescoplc.com/media/n3jdzlzc/2138002p5rnkc5w2jz46-2024-02-24-t01_preview.xhtml'
response = requests.get(url) # <Response [200]> => pas d'erreur

if response.ok: # si on a Response [200]
  soup = BeautifulSoup(response.text , 'lxml')
  title = soup.find('title')
  print(title)


<title></title>


In [ ]:
# Charger le contenu HTML
url = 'https://www.tescoplc.com/media/n3jdzlzc/2138002p5rnkc5w2jz46-2024-02-24-t01_preview.xhtml'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Étape 1 : Trouver le conteneur principal (probablement classe `pf`)
table_container = soup.find('div', class_='pf')

# Étape 2 : Trouver toutes les lignes du tableau (probablement classe `pc`)
rows = table_container.find_all('div', class_='pc')

# Étape 3 : Extraire les cellules dans chaque ligne (probablement classe `pi`)
data = []
for row in rows:
    cells = row.find_all('div', class_='pi')
    row_data = [cell.text.strip() for cell in cells]  # Nettoyer les données
    data.append(row_data)

# Étape 4 : Convertir en DataFrame Pandas
df = pd.DataFrame(data)

# Afficher le résultat
print(df)


Empty DataFrame
Columns: []
Index: [0]


In [1]:
# URL de la page
url = "https://www.tescoplc.com/media/n3jdzlzc/2138002p5rnkc5w2jz46-2024-02-24-t01_preview.xhtml"

# Envoyer la requête HTTP
response = requests.get(url)
response.raise_for_status()  # Vérifie si la requête est réussie

# Charger le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# Trouver le conteneur principal
page_container = soup.find("div", id="page-container")

# Trouver toutes les pages dans le conteneur
pages = page_container.find_all("div", class_="pf")

# Itérer sur chaque page et extraire le contenu
for page in pages:
    page_id = page.get("id")  # Identifier la page (par exemple, 'pf1')
    print(f"\n--- Contenu de la page : {page_id} ---")
    print(page.get_text(strip=True))  # Extraire tout le texte de la page


In [ ]:
# Trouver la page avec l'ID "pf18"
page_pf18 = None
for page in pages:
    page_id = page.get("id")  # Identifier la page (par exemple, 'pf1', 'pf18', etc.)
    if page_id == "pf18":
        page_pf18 = page
        break

# Vérifier si la page pf18 a été trouvée
if page_pf18:
    print("--- Contenu de la page pf18 ---")
    print(page_pf18.get_text(strip=True))  # Afficher le texte brut de la page pf18

--- Contenu de la page pf18 ---
Financial review52 weeks ended 24 February 20242,7FY 23/24FY 22/233Change atactualratesChange atconstant ratesSales (exc. VAT, exc. fuel)4£61,477m£57,216m7.4%7.2%Fuel£6,710m£8,106m(17.2)%(17.2)%Revenue (exc. VAT, inc. fuel)£68,187m£65,322m4.4%4.2%Adjusted operating profit5£2,829m£2,509m12.8%12.7%Adjusting items£(8)m£(1,099)mStatutory operatingprofit£2,821m£1,410m100.1%Net finance costs£(538)m£(536)mJoint ventures and associates£6m£8mStatutory profit before tax£2,289m£882m159.5%Group tax£(525)m£(224)mStatutory profit after tax£1,764m£658m168.1%Adjusted diluted EPS523.41p20.53p14.0%Statutory diluted EPS24.53p8.81p178.4%Dividend per share12.10p10.90p11.0%Net debt6,7£(9,764)m£(10,493)m6.9%Retail free cash flow6£2,063m£2,133m(3.3)%Capex9£1,314m£1,235m6.4%1.Following the announcement in February 2024that wehave reached anagreement tosell our banking operations, theperformanceof these banking operationshas been presented as adiscontinued operationwith comparati

In [ ]:
# URL de la page
url = "https://www.tescoplc.com/media/n3jdzlzc/2138002p5rnkc5w2jz46-2024-02-24-t01_preview.xhtml"

# Envoyer la requête HTTP
response = requests.get(url)
response.raise_for_status()  # Vérifie si la requête est réussie

# Charger le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# Trouver la page avec l'ID "pf18"
page_pf18 = soup.find("div", id="pf18")

if page_pf18:
    print("--- Contenu de la page pf18 ---")

    # Trouver tous les divs correspondant à des lignes ou parties de lignes du tableau
    rows = page_pf18.find_all("div", class_="t")

    # Extraire et afficher les contenus visibles dans chaque div
    table_data = []
    for row in rows:
        # Extraire tout le texte brut dans chaque div
        row_text = row.get_text(strip=True)
        if row_text:  # On n'ajoute que le texte non vide
            table_data.append(row_text)

    # Afficher le contenu extrait
    print("\n--- Contenu extrait du tableau ---")
    if table_data:
        for line in table_data:
            print(line)
    else:
        print("Aucun texte pertinent trouvé.")
else:
    print("La page pf18 n'a pas été trouvée.")


--- Contenu de la page pf18 ---

--- Contenu extrait du tableau ---
Financial review
52 weeks ended 24 February 2024
2,7
FY 23/24FY 22/23
3
Change at
actualrates
Change at
constant rates
Sales (exc. VAT, exc. fuel)
4
£61,477m£57,216m7.4%7.2%
Fuel£6,710m£8,106m(17.2)%(17.2)%
Revenue (exc. VAT, inc. fuel)£68,187m£65,322m4.4%4.2%
Adjusted operating profit
5
£2,829m£2,509m12.8%12.7%
Adjusting items£(8)m£(1,099)m
Statutory operatingprofit£2,821m£1,410m100.1%
Net finance costs£(538)m£(536)m
Joint ventures and associates£6m£8m
Statutory profit before tax£2,289m£882m159.5%
Group tax£(525)m£(224)m
Statutory profit after tax£1,764m£658m168.1%
Adjusted diluted EPS
5
23.41p20.53p14.0%
Statutory diluted EPS24.53p8.81p178.4%
Dividend per share12.10p10.90p11.0%
Net debt
6,7
£(9,764)m£(10,493)m6.9%
Retail free cash flow
6
£2,063m£2,133m(3.3)%
Capex
9
£1,314m£1,235m6.4%
1.Following the announcement in February 2024that wehave reached anagreement tosell our banking operations, theperformanceof these ban

In [ ]:
# URL de la page
url = "https://www.tescoplc.com/media/n3jdzlzc/2138002p5rnkc5w2jz46-2024-02-24-t01_preview.xhtml"

# Envoyer la requête HTTP
response = requests.get(url)
response.raise_for_status()  # Vérifie si la requête est réussie

# Charger le contenu HTML avec BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# Trouver la page avec l'ID "pf18"
page_pf18 = soup.find("div", id="pf18")

if page_pf18:
    print("--- Contenu de la page pf18 ---")

    # Trouver tous les divs correspondant à des lignes ou parties de lignes du tableau
    rows = page_pf18.find_all("div", class_="t")

    # Groupement des données en lignes
    table_data = []
    current_row = []  # Temporaire pour contenir une ligne

    for row in rows:
        # Extraire le texte brut de chaque div
        row_text = row.get_text(strip=True)

        if row_text:
            current_row.append(row_text)

        # Lorsqu'une ligne est complète, ajouter la ligne à table_data
        # La condition de fin de ligne pourrait dépendre d'un nombre fixe de colonnes ou de caractéristiques du texte
        if len(current_row) == 4:  # Par exemple, on suppose qu'une ligne a 4 colonnes
            table_data.append(current_row)
            current_row = []  # Réinitialiser pour la prochaine ligne

    # Si des données sont encore dans current_row (dernière ligne incomplète), on les ajoute aussi
    if current_row:
        table_data.append(current_row)

    # Afficher le tableau reconstruit
    print("\n--- Tableau reconstruit ---")
    for line in table_data:
        print(line)

else:
    print("La page pf18 n'a pas été trouvée.")


--- Contenu de la page pf18 ---

--- Tableau reconstruit ---
['Financial review', '52 weeks ended 24 February 2024', '2,7', 'FY 23/24FY 22/23']
['3', 'Change at', 'actualrates', 'Change at']
['constant rates', 'Sales (exc. VAT, exc. fuel)', '4', '£61,477m£57,216m7.4%7.2%']
['Fuel£6,710m£8,106m(17.2)%(17.2)%', 'Revenue (exc. VAT, inc. fuel)£68,187m£65,322m4.4%4.2%', 'Adjusted operating profit', '5']
['£2,829m£2,509m12.8%12.7%', 'Adjusting items£(8)m£(1,099)m', 'Statutory operatingprofit£2,821m£1,410m100.1%', 'Net finance costs£(538)m£(536)m']
['Joint ventures and associates£6m£8m', 'Statutory profit before tax£2,289m£882m159.5%', 'Group tax£(525)m£(224)m', 'Statutory profit after tax£1,764m£658m168.1%']
['Adjusted diluted EPS', '5', '23.41p20.53p14.0%', 'Statutory diluted EPS24.53p8.81p178.4%']
['Dividend per share12.10p10.90p11.0%', 'Net debt', '6,7', '£(9,764)m£(10,493)m6.9%']
['Retail free cash flow', '6', '£2,063m£2,133m(3.3)%', 'Capex']
['9', '£1,314m£1,235m6.4%', '1.Following the 

In [ ]:
!pip install pytesseract pillow pandas

In [ ]:
from google.colab import files

# Téléchargement de l'image depuis votre machine
uploaded = files.upload()  # Cela ouvre une interface pour charger un fichier

Saving tableaux sales.PNG to tableaux sales.PNG


In [ ]:
# Charger l'image uploadée
image_path = list(uploaded.keys())[0]
img = Image.open(image_path)

# Étape 2 : Appliquer Tesseract OCR pour extraire le texte
extracted_text = pytesseract.image_to_string(img)

# Afficher le texte brut extrait
print("Texte extrait :")
print(extracted_text)

Texte extrait :
 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

Change at Change at
52 weeks ended 24 February 20247 FY 23/24 FY22/2 __actualrates constant rates
Sales (exc. VAT, exc. fuel)* £61,477m ‘£57,216 7.4% 7.2%
Fuel £6710m £8,106m (7.21% (7.21%
Revenue (exc. VAT, inc. fuel) £68,187 £65,322m 44% 4.2%
Adjusted operating profit £2,829m ——_-£2,509m 12.8% 127%
Adjusting items £18)m___£{1,099)m

Statutory operating profit £2,82Im___‘£1,410m 100.1%

Net finance costs £(538)m £1536),

Joint ventures and associates £6m 28m

Statutory profit before tax £2,289m £882m 159.5%

Group tax 8(525)m 8224)

Statutory profit after tax £1,764m £658m 168.1%

Adjusted diluted EPS 23.81p 2053p 14.0%

Statutory diluted EPS 24.53p 88ip 178.4%

Dividend per share 1210p 10.90p 11.0%

Net debt” £(9,764)m —_ £(10,493)m 6.9%

Retail free cash flow* £2,063m ——_—£2,133m 3.31%

Capex £1,314m_—_-£1,235m 6.4%

 



In [ ]:
# Étape 3 : Prétraiter et transformer le texte en un DataFrame
def text_to_dataframe(extracted_text):
    # Diviser le texte en lignes
    lines = extracted_text.split("\n")

    # Nettoyer les lignes vides
    lines = [line.strip() for line in lines if line.strip()]

    # Supprimer les lignes inutiles si nécessaire (exemple avec entêtes non souhaitées)
    # lines = lines[2:]  # Supposons que les 2 premières lignes sont des en-têtes inutiles

    # Diviser chaque ligne en colonnes (adapter au séparateur réel : tabulations, espaces multiples, etc.)
    data = [line.split() for line in lines]

    # Transformer en DataFrame
    df = pd.DataFrame(data)
    return df

# Transformer le texte brut en DataFrame
df = text_to_dataframe(extracted_text)

# Étape 4 : Afficher le DataFrame
print("\nDataFrame reconstruit :")
df



DataFrame reconstruit :


,0,1,2,3,4,5,6,7,8,9,10,11
0,Change,at,Change,at,None,None,None,None,None,None,None,None
1,52,weeks,ended,24,February,20247,FY,23/24,FY22/2,__actualrates,constant,rates
2,Sales,(exc.,"VAT,",exc.,fuel)*,"£61,477m","‘£57,216",7.4%,7.2%,None,None,None
3,Fuel,£6710m,"£8,106m",(7.21%,(7.21%,None,None,None,None,None,None,None
4,Revenue,(exc.,"VAT,",inc.,fuel),"£68,187","£65,322m",44%,4.2%,None,None,None
5,Adjusted,operating,profit,"£2,829m","——_-£2,509m",12.8%,127%,None,None,None,None,None
6,Adjusting,items,"£18)m___£{1,099)m",None,None,None,None,None,None,None,None,None
7,Statutory,operating,profit,"£2,82Im___‘£1,410m",100.1%,None,None,None,None,None,None,None
8,Net,finance,costs,£(538)m,"£1536),",None,None,None,None,None,None,None
9,Joint,ventures,and,associates,£6m,28m,None,None,None,None,None,None


In [ ]:
# Charger l'image avec OpenCV
img_cv = cv2.imread(image_path)

# Convertir en niveaux de gris
gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

# Appliquer un seuillage binaire pour améliorer le contraste
_, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# Sauvegarder l'image prétraitée pour vérifier
cv2.imwrite("preprocessed_image.png", binary)

# Relancer l'OCR sur l'image prétraitée
preprocessed_img = Image.open("preprocessed_image.png")
extracted_text = pytesseract.image_to_string(preprocessed_img)

print("Texte extrait après prétraitement :")
print(extracted_text)

Texte extrait après prétraitement :
 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

‘Change at Change at
fod weeks ended 24 February 20247 fy 2a/2a fyz2/2s _actualrates constant rates
Sales texc. VAT, exc. fuell* €61,477m ——£57,216m 7% 7.2%
Fuel £6710m £8106m (7.29% 7.286
Revenue (exc. VAT, inc. fuel) 268,187m___£65,322m 4A% 4%
‘Adjusted operating profit €2,629m _-€2,508m 12.8% 7%
Adjusting terms 28m £0,099}

Statutory operating profit €2,821m e1,410m 001%

Net finance casts (538) £(536)m

Joint ventures and associates 6m £8m

Statutory profit before tax £2,289m €882m 159.5%

Group tax £6253 £224)

Statutory profit after tax £1,764m 658m 168.1%

Adjusted diluted EP 2281p 2053p 14.0%

Statutory diluted EPS 2453p 8381p 178.4%

Dividend per share 1210p 10.90p 1.0%

Net debt” £18,764) __£{10,493}m 6.9%

Retail free cash flow €2,063m «£2,133 3.3%

Capet e13i4m__£1,235m 6.4%

 



In [ ]:
def parse_table_data(extracted_text):
    # Diviser le texte en lignes
    lines = extracted_text.split("\n")

    # Séparer chaque ligne en colonnes (en fonction de l'espacement ou des tabulations)
    table = []
    for line in lines:
        columns = line.split()  # Adapter si nécessaire (par exemple, pour des séparateurs spécifiques)
        if columns:
            table.append(columns)

    return table

# Reconstituer le tableau
table = parse_table_data(extracted_text)

# Afficher le tableau reconstruit
for row in table:
    print(row)

['‘Change', 'at', 'Change', 'at']
['fod', 'weeks', 'ended', '24', 'February', '20247', 'fy', '2a/2a', 'fyz2/2s', '_actualrates', 'constant', 'rates']
['Sales', 'texc.', 'VAT,', 'exc.', 'fuell*', '€61,477m', '——£57,216m', '7%', '7.2%']
['Fuel', '£6710m', '£8106m', '(7.29%', '7.286']
['Revenue', '(exc.', 'VAT,', 'inc.', 'fuel)', '268,187m___£65,322m', '4A%', '4%']
['‘Adjusted', 'operating', 'profit', '€2,629m', '_-€2,508m', '12.8%', '7%']
['Adjusting', 'terms', '28m', '£0,099}']
['Statutory', 'operating', 'profit', '€2,821m', 'e1,410m', '001%']
['Net', 'finance', 'casts', '(538)', '£(536)m']
['Joint', 'ventures', 'and', 'associates', '6m', '£8m']
['Statutory', 'profit', 'before', 'tax', '£2,289m', '€882m', '159.5%']
['Group', 'tax', '£6253', '£224)']
['Statutory', 'profit', 'after', 'tax', '£1,764m', '658m', '168.1%']
['Adjusted', 'diluted', 'EP', '2281p', '2053p', '14.0%']
['Statutory', 'diluted', 'EPS', '2453p', '8381p', '178.4%']
['Dividend', 'per', 'share', '1210p', '10.90p', '1.0%']